In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import subprocess
from datetime import date
import re
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth',500)
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from importlib import reload



In [ ]:
## EDIT THESE VARIABLES
analysis_version = "2017_10_19"
project_dir = Path('/Users/rodgersleejg/data/hpc/NNDSP') # needs to be pathlib.Path object

In [ ]:
bids_dir = project_dir.joinpath('bids_2017_07_14')

# conf_script = mriqc_dir.joinpath('conf' + analysis_version + '.sh')

mriqc_dir  = project_dir.joinpath('anal/mriqc_files/other_files')
if not mriqc_dir.exists():
    mriqc_dir.mkdir()
output_folder =  project_dir / 'derivatives' / 'mriqc'
if not output_folder.exists():
    output_folder.mkdir()
classifier_output =  output_folder.joinpath('classifier')
if not classifier_output.exists():
    classifier_output.mkdir()
base_work_dir = output_folder.joinpath('work')
if not base_work_dir.exists():
    base_work_dir.mkdir()
log_dir = mriqc_dir.joinpath('swarm_output_' +  analysis_version)
if not log_dir.exists():
    log_dir.mkdir()
manual_qc = output_folder.joinpath('manual_qc_round_2.tsv')
# swarm_path = mriqc_dir.joinpath('mriqc_' + analysis_version + '.cmd')


df_qc_full_pkl = Path('anal/mriqc_files/other_files/qc_pickle_for_v2_exploration.pklz')
mriqc_with_predictions = Path('derivatives/mriqc/with_mriqc_predictions.csv')
plottable_data = Path('derivatives/mriqc/classifer_plot_data.pklz')
plottable_data_euler = Path('derivatives/mriqc/classifer_plot_data_euler.pklz')

In [ ]:
%pwd
%cd {project_dir}
%pwd

In [ ]:
import anal.python_modules.inner_merge_and_report as pd_custom

In [ ]:
df_qc_full = pd.read_pickle(df_qc_full_pkl)

In [ ]:
len(df_qc_full.groupby(['MASKID','run']).count())

# Using Euler number for classification 

In [ ]:
def get_lr_surface_holes(aseg_path):
    pat = re.compile('.*holes in [lr]h surfaces prior to fixing, (\d*).*')
    vals = [int(x) for x in pat.findall(aseg_path.read_text())]
    return vals
def get_maskid_from_aseg_path(aseg_path):
    pat = re.compile('.*sub-(?P<subject_num>\d{2,4})_.*/.*/.*')
    vals = [int(x) for x in pat.findall(aseg_path.as_posix())]
    assert len(vals) ==1
    sub_num = vals[0]
    return '{n:04d}'.format(n=sub_num)

def get_run_from_aseg_path(aseg_path):
    pat = re.compile('.*sub-\d{2,4}_run-(?P<run>\d{2,4})/.*/.*')
    vals = [int(x) for x in pat.findall(aseg_path.as_posix())]
    assert len(vals) ==1
    run = vals[0]
    return 'run-' + '{n:03d}'.format(n=run)

# aseg_path = Path('/data/NNDSP/derivatives/fs5.3_subj/sub-001/stats/aseg.stats')

In [ ]:
df_aseg = pd.DataFrame({'aseg_path': list(Path('derivatives/fs_subj_john/').glob('sub*/stats/aseg.stats'))})
df_aseg = (df_aseg.
           assign(euler = lambda df: df.aseg_path.apply(get_lr_surface_holes)).
           assign(mean_euler = lambda df: df.euler.apply(np.mean))
          )

In [ ]:
df_aseg['MASKID'] = df_aseg.aseg_path.apply(get_maskid_from_aseg_path)
df_aseg['run'] = df_aseg.aseg_path.apply(get_run_from_aseg_path)

df_aseg = df_aseg.assign(subject = 'sub-' + df_aseg.MASKID)
df_aseg.head()

In [ ]:
len(df_aseg)

In [ ]:
df_euler = df_qc_full.drop('_merge',axis = 1, errors = 'ignore').copy().merge(df_aseg, on = ['MASKID','run'],how = 'outer',indicator = True)
df_euler._merge.value_counts()

In [ ]:
df_euler = df_euler.query("_merge == 'both'")

In [ ]:
df_euler.head()['mean_euler']

In [ ]:
df_euler['prob_y'] = df_euler['mean_euler']

### Get performance metrics for the classifier (euler predicting manual)

In [ ]:
from anal.python_modules import classification
reload(classification)

In [ ]:
from IPython.core.debugger import Pdb; ipdb=Pdb()

In [ ]:
manual_metrics = ['Freesurfer_avg_ext_rating', 'Freesurfer_avg_int_rating', 'MPRAGE']
classifier_metrics = ['tpr','fpr','fdr','fp','tp','fn','tn']
for metric in manual_metrics:
    col_prob = 'prob_y'
    col_true = metric + '_thresholded'
    threshold = 3
    df_euler[col_true] = df_euler[metric] >= threshold
    df_performance = classification.get_classification_scores(df_euler,col_true,col_prob)
#     ipdb.runcall(classification.get_classification_scores,df_euler,col_true,col_prob)
    df_euler.drop(df_performance.columns,axis = 1, inplace=True,errors='ignore')
    df_euler = pd.concat([df_euler, df_performance],axis = 1)
df_euler.head()

### Gather value columns together using melt and create labels

Value cols need to all be the same type so that they can be melted to a single columns

In [ ]:
t_cols = df_euler.filter(regex = '^(Free|MP).*(' + 'thresholded' + ')').columns
df_euler.loc[:,t_cols] = df_euler.loc[:,t_cols].apply(lambda col:col.astype(float),axis = 0)

tail_of_regex = '|'.join(classifier_metrics) + '|thresholded'
cols_regex = '^(Free|MP).*(' + tail_of_regex + ')'
value_cols = df_euler.filter(regex= cols_regex, axis=1).columns
ids_to_keep = pd.Index(['MASKID','run','prob_y', 'pred_y','threshold'])

print('Regex for value columns to be melted,separated and pivoted: ', cols_regex)
print('ids: ',ids_to_keep,'\n\n\nvalues: ',value_cols)

In [ ]:
df_melted = df_euler.melt(id_vars = ids_to_keep,
                var_name= 'binarized_manual_qc_scores',
                            value_name= 'value',
                value_vars= value_cols)
df_melted = (
    pd.concat(
        [df_melted,
        (df_melted.
         binarized_manual_qc_scores.
         str.
         extract(expand=True,
                 pat= '(?P<manual_qc_type>.*)_(?P<value_type>' + tail_of_regex  + ')')
        )],
    axis = 1)
)
df_melted = df_melted.loc[df_melted.MASKID.notnull(),:]
len(df_melted)

### Create a column each for the tpr and fpr variables

In [ ]:
cols = ['MASKID','run', 'manual_qc_type','prob_y','pred_y','value_type']
df_roc_euler = df_melted[[*cols,'value']].set_index(cols).unstack().reset_index()

In [ ]:
cols_from_pivot = df_roc_euler.columns.levels[1][:-1]
df_roc_euler.head()

df_roc_euler.columns = [*cols[:-1], *cols_from_pivot]
df_roc_euler['fpratio'] = df_roc_euler.fp/df_roc_euler.tp
df_roc_euler['positive'] = df_roc_euler.fp + df_roc_euler.tp
df_roc_euler.to_pickle(plottable_data_euler)
df_roc_euler.head()

In [ ]:
print((len(df_roc_euler.query('manual_qc_type == "MPRAGE"')), plottable_data_euler))